In [2]:
from pymongo import MongoClient
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from folium import Marker
from shapely.geometry import Point
from shapely.ops import nearest_points
import numpy as np

def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

db, coll = connectCollection('companies','companies')

pipeline = [
    { "$unwind": "$offices"},    
 ]
results = list(coll.aggregate(pipeline))


In [3]:
#creating DataFrame

df = pd.DataFrame(results)
df

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,video_embeds,screenshots,external_links,partners,founded_month,founded_day,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,"[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],9.0,15.0,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],10.0,17.0,NaN,NaN,NaN,NaN
3,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],10.0,17.0,NaN,NaN,NaN,NaN
4,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,[],[],[],[],6.0,2.0,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16700,52cdef7f4bab8bd67529c6f5,Oriact,oriact,http://www.crunchbase.com/company/oriact,http://www.oriact.com,,,,software,NaN,...,[],[],"[{'external_url': 'http://origo.ethz.ch', 'tit...",[],4.0,3.0,NaN,NaN,None,None
16701,52cdef7f4bab8bd67529c6f6,Adhunk,adhunk,http://www.crunchbase.com/company/adhunk,http://www.adhunk.com,http://blog.adhunk.com,http://blog.adhunk.com/feed,,advertising,3.0,...,[],[],[{'external_url': 'http://www.hubpages.in/inte...,[],NaN,NaN,NaN,NaN,None,None
16702,52cdef7f4bab8bd67529c6f7,AfterLogic,afterlogic,http://www.crunchbase.com/company/afterlogic,http://www.afterlogic.com,,,afterlogic,software,NaN,...,[],"[{'available_sizes': [[[150, 137], 'assets/ima...",[],[],NaN,NaN,NaN,NaN,None,None
16703,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,entegreat-solutions,http://www.crunchbase.com/company/entegreat-so...,,,,,software,NaN,...,[],[],[],[],NaN,NaN,NaN,NaN,None,None


In [4]:
#dropping deadpooled companies
print(df.shape)
df = df[df.deadpooled_year.isnull()]
df.reset_index(inplace=True, drop= True)
print(df.shape)
df.head()


(16705, 42)
(15848, 42)


,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,video_embeds,screenshots,external_links,partners,founded_month,founded_day,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,[],[],[],[],6.0,2.0,NaN,NaN,None,None
1,52cdef7c4bab8bd675297d91,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,...,"[{'embed_code': '<object width=""425"" height=""3...",[],[],[],6.0,1.0,NaN,NaN,None,None
2,52cdef7c4bab8bd675297d92,Flektor,flektor,http://www.crunchbase.com/company/flektor,http://www.flektor.com,http://www.flektor-blog.com,http://www.flektor-blog.com/video_editing_soft...,None,games_video,NaN,...,[],[],[],[],NaN,NaN,NaN,NaN,None,None
3,52cdef7c4bab8bd675297d93,Fox Interactive Media,fox-interactive-media,http://www.crunchbase.com/company/fox-interact...,http://www.newscorp.com,,,twitterapi,web,0.0,...,"[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],6.0,1.0,NaN,NaN,,None
4,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,"[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[117, 150], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[],10.0,11.0,NaN,NaN,None,None


In [5]:
df.offices[7]

{'description': '',
 'address1': '',
 'address2': '',
 'zip_code': '',
 'city': 'New York City',
 'state_code': 'NY',
 'country_code': 'USA',
 'latitude': None,
 'longitude': None}

In [6]:
df = pd.concat([df.drop(['offices'], axis=1), df['offices'].apply(pd.Series)], axis=1)
# df = pd.concat([df.drop(['offices'], axis=1), pd.DataFrame(df.offices.tolist())], axis=1)

In [7]:
display(df.head(10))
df.index

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,None,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573
1,52cdef7c4bab8bd675297d91,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,...,None,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
2,52cdef7c4bab8bd675297d92,Flektor,flektor,http://www.crunchbase.com/company/flektor,http://www.flektor.com,http://www.flektor-blog.com,http://www.flektor-blog.com/video_editing_soft...,None,games_video,NaN,...,None,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768
3,52cdef7c4bab8bd675297d93,Fox Interactive Media,fox-interactive-media,http://www.crunchbase.com/company/fox-interact...,http://www.newscorp.com,,,twitterapi,web,0.0,...,None,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.076179,-118.394170
4,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,None,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
5,52cdef7c4bab8bd675297d96,Gizmoz,gizmoz,http://www.crunchbase.com/company/gizmoz,http://gizmoz.com,,,None,web,NaN,...,None,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472
6,52cdef7c4bab8bd675297d95,StumbleUpon,stumbleupon,http://www.crunchbase.com/company/stumbleupon,http://www.stumbleupon.com,http://www.stumbleupon.com/blog,,stumbleupon,web,NaN,...,None,,,,,San Francisco,CA,USA,37.775196,-122.419204
7,52cdef7c4bab8bd675297d95,StumbleUpon,stumbleupon,http://www.crunchbase.com/company/stumbleupon,http://www.stumbleupon.com,http://www.stumbleupon.com/blog,,stumbleupon,web,NaN,...,None,,,,,New York City,NY,USA,NaN,NaN
8,52cdef7c4bab8bd675297d97,Scribd,scribd,http://www.crunchbase.com/company/scribd,http://scribd.com,http://blog.scribd.com/,http://blog.scribd.com/feed/,scribd,news,50.0,...,None,HQ,539 Bryant Street,,94107,San Francisco,CA,USA,37.789634,-122.404052
9,52cdef7c4bab8bd675297d98,Slacker,slacker,http://www.crunchbase.com/company/slacker,http://www.slacker.com,,,SlackerRadio,music,NaN,...,None,,16935 W. Bernardo Dr. Suite 101,,92127,San Diego,CA,USA,33.022176,-117.081406


RangeIndex(start=0, stop=15848, step=1)

In [8]:
# Dropping rows for which we don't have coordinates
drop_null_coords = list(df[((df.latitude.isnull() == True) | (df.longitude.isnull() == True))].index)
df = df.drop(drop_null_coords)
print(df.shape)
df.head()

(10165, 50)


,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,None,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573
1,52cdef7c4bab8bd675297d91,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,...,None,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
2,52cdef7c4bab8bd675297d92,Flektor,flektor,http://www.crunchbase.com/company/flektor,http://www.flektor.com,http://www.flektor-blog.com,http://www.flektor-blog.com/video_editing_soft...,None,games_video,NaN,...,None,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768
3,52cdef7c4bab8bd675297d93,Fox Interactive Media,fox-interactive-media,http://www.crunchbase.com/company/fox-interact...,http://www.newscorp.com,,,twitterapi,web,0.0,...,None,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.076179,-118.394170
4,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,None,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523


In [9]:
df.category_code.unique()

array(['web', 'games_video', 'news', 'music', 'mobile', 'social',
       'search', 'advertising', 'security', 'photo_video', 'finance',
       'network_hosting', 'software', 'ecommerce', 'hardware',
       'enterprise', 'travel', 'public_relations', 'other', 'real_estate',
       'messaging', 'semiconductor', 'analytics', 'legal', 'health',
       'sports', 'biotech', 'cleantech', 'education', 'consulting',
       'hospitality', 'fashion', 'nonprofit', 'nanotech', 'automotive',
       None, 'design', 'manufacturing', 'transportation', 'local',
       'medical'], dtype=object)

In [10]:
df[df.category_code.isnull() == True]

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
1378,52cdef7c4bab8bd675298276,SpaceTime,spacetime,http://www.crunchbase.com/company/spacetime,http://www.spacetime.com,,,None,None,NaN,...,None,,576 Fifth Avenue,,10036,New York,NY,USA,40.756662,-73.979038
1384,52cdef7c4bab8bd67529827e,Repeater Store,repeater-store,http://www.crunchbase.com/company/repeater-store,http://www.repeaterstore.com,http://repeaterstore.com/news,,,None,NaN,...,None,,26081 Merit Circle,Unit 119,92653,Laguna Hills,CA,USA,33.574054,-117.674526
1419,52cdef7c4bab8bd6752982ab,Pyra Labs,pyra-labs,http://www.crunchbase.com/company/pyra-labs,,,,,None,NaN,...,None,,,,,San Francisco,CA,USA,37.775196,-122.419204
1473,52cdef7c4bab8bd6752982e0,NuvoMedia,nuvomedia,http://www.crunchbase.com/company/nuvomedia,None,None,None,None,None,NaN,...,None,,,,,Mountain View,CA,USA,37.400465,-122.073003
1491,52cdef7c4bab8bd6752982f8,A1Vacations,a1vacations,http://www.crunchbase.com/company/a1vacations,http://www.a1vacations.com,,http://www.homeaway.com/info/media-center,homeaway,None,NaN,...,None,,95 Westlake Rd.,Suite 205,24101,Hardy,VA,USA,37.129183,-79.750646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15620,52cdef7f4bab8bd67529c5d1,iTradeNetwork,itradenetwork,http://www.crunchbase.com/company/itradenetwork,http://www.itradenetwork.com,,,ITNteam,None,NaN,...,None,,4155 Hopyard Rd,Suite 10,94588,Pleasanton,CA,USA,37.685803,-121.901057
15755,52cdef7f4bab8bd67529c686,Email Data Source,email-data-source,http://www.crunchbase.com/company/email-data-s...,http://www.emaildatasource.com,,,,None,NaN,...,None,,"65 Broadway, Suite 601",,10006,New York,NY,USA,40.707158,-74.012163
15802,52cdef7f4bab8bd67529c6c3,Advanced Control Components,advanced-control-components,http://www.crunchbase.com/company/advanced-con...,,,,,None,NaN,...,None,,611 Industrial Way Eatontown,,07724,,NJ,USA,40.270986,-74.069035
15808,52cdef7f4bab8bd67529c6c8,Nellix,nellix,http://www.crunchbase.com/company/nellix,http://www.nellix.com,,,,None,NaN,...,None,,2465-B Faber Place,,94303,Palo Alto,CA,USA,37.449972,-122.116178


In [11]:
drop_null_category = list(df[(df.category_code.isnull() == True)].index)
df = df.drop(drop_null_category)
df

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,None,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573
1,52cdef7c4bab8bd675297d91,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,...,None,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
2,52cdef7c4bab8bd675297d92,Flektor,flektor,http://www.crunchbase.com/company/flektor,http://www.flektor.com,http://www.flektor-blog.com,http://www.flektor-blog.com/video_editing_soft...,None,games_video,NaN,...,None,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768
3,52cdef7c4bab8bd675297d93,Fox Interactive Media,fox-interactive-media,http://www.crunchbase.com/company/fox-interact...,http://www.newscorp.com,,,twitterapi,web,0.0,...,None,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.076179,-118.394170
4,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,None,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15842,52cdef7f4bab8bd67529c6f1,Willdan Group,willdan-group,http://www.crunchbase.com/company/willdan-group,http://www.willdan.com,,,,other,385.0,...,None,,2401 East Katella Avenue,Suite 300,92806-5909,Anaheim,CA,USA,33.806525,-117.882314
15843,52cdef7f4bab8bd67529c6f5,Oriact,oriact,http://www.crunchbase.com/company/oriact,http://www.oriact.com,,,,software,NaN,...,None,,LÃ¤rchenweg 7,,6403,KÃ¼ssnacht,None,CHE,47.088219,8.437163
15845,52cdef7f4bab8bd67529c6f7,AfterLogic,afterlogic,http://www.crunchbase.com/company/afterlogic,http://www.afterlogic.com,,,afterlogic,software,NaN,...,None,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554
15846,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,entegreat-solutions,http://www.crunchbase.com/company/entegreat-so...,,,,,software,NaN,...,None,,,,,Birmingham,AL,USA,33.518885,-86.816068


In [12]:
df.category_code.unique()

array(['web', 'games_video', 'news', 'music', 'mobile', 'social',
       'search', 'advertising', 'security', 'photo_video', 'finance',
       'network_hosting', 'software', 'ecommerce', 'hardware',
       'enterprise', 'travel', 'public_relations', 'other', 'real_estate',
       'messaging', 'semiconductor', 'analytics', 'legal', 'health',
       'sports', 'biotech', 'cleantech', 'education', 'consulting',
       'hospitality', 'fashion', 'nonprofit', 'nanotech', 'automotive',
       'design', 'manufacturing', 'transportation', 'local', 'medical'],
      dtype=object)

In [13]:
tech = ["web", "games_video", "mobile", "social", "photo_video", "network_hosting", "software", "ecommerce", "hardware", "semiconductor", "analytics", "biotech", "cleantech", "nanotech"]
df["Tech/Other"] = df["category_code"].apply(lambda x: "Tech" if x in tech else "Other")
df

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,Tech/Other
0,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573,Tech
1,52cdef7c4bab8bd675297d91,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,...,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,Tech
2,52cdef7c4bab8bd675297d92,Flektor,flektor,http://www.crunchbase.com/company/flektor,http://www.flektor.com,http://www.flektor-blog.com,http://www.flektor-blog.com/video_editing_soft...,None,games_video,NaN,...,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768,Tech
3,52cdef7c4bab8bd675297d93,Fox Interactive Media,fox-interactive-media,http://www.crunchbase.com/company/fox-interact...,http://www.newscorp.com,,,twitterapi,web,0.0,...,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.076179,-118.394170,Tech
4,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15842,52cdef7f4bab8bd67529c6f1,Willdan Group,willdan-group,http://www.crunchbase.com/company/willdan-group,http://www.willdan.com,,,,other,385.0,...,,2401 East Katella Avenue,Suite 300,92806-5909,Anaheim,CA,USA,33.806525,-117.882314,Other
15843,52cdef7f4bab8bd67529c6f5,Oriact,oriact,http://www.crunchbase.com/company/oriact,http://www.oriact.com,,,,software,NaN,...,,LÃ¤rchenweg 7,,6403,KÃ¼ssnacht,None,CHE,47.088219,8.437163,Tech
15845,52cdef7f4bab8bd67529c6f7,AfterLogic,afterlogic,http://www.crunchbase.com/company/afterlogic,http://www.afterlogic.com,,,afterlogic,software,NaN,...,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554,Tech
15846,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,entegreat-solutions,http://www.crunchbase.com/company/entegreat-so...,,,,,software,NaN,...,,,,,Birmingham,AL,USA,33.518885,-86.816068,Tech


In [14]:
df.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'deadpooled_year', 'tag_list',
       'alias_list', 'email_address', 'phone_number', 'description',
       'created_at', 'updated_at', 'overview', 'image', 'products',
       'relationships', 'competitions', 'providerships', 'total_money_raised',
       'funding_rounds', 'investments', 'acquisition', 'acquisitions',
       'milestones', 'video_embeds', 'screenshots', 'external_links',
       'partners', 'founded_month', 'founded_day', 'deadpooled_month',
       'deadpooled_day', 'deadpooled_url', 'ipo', 'description', 'address1',
       'address2', 'zip_code', 'city', 'state_code', 'country_code',
       'latitude', 'longitude', 'Tech/Other'],
      dtype='object')

In [15]:
import re
def moneyRaise(value):
    dicc_coin = {'CAD': 1.3124497992,'RUB': 63.7561301828, 'EUR': 0.8916629514, 'GBP': 0.7991529202}
    values_money = {'K':1000, 'M':1000000, 'B': 100000000000}
    value_number = float(re.search('[+-]?([0-9]*[.])?[0-9]+', value)[0])
    if value.endswith('B'):
        exchange = value_number*(values_money['B'])
    elif value.endswith('K[k]'):
        exchange = value_number*(values_money['K'])
    elif value.endswith('M'):
        exchange = value_number*(values_money['M'])
    elif value.startswith("C"):
        exchange =  value_number*(dicc_coin['CAD'])
    elif value.startswith("$"):
        exchange =  value_number
    elif value.startswith("€"):
        exchange = value_number*(dicc_coin['EUR'])
    elif value.startswith("£"):
        exchange = value_number*(dicc_coin['GBP'])
    elif value.startswith("r"):
        exchange = value_number*(dicc_coin['RUB'])
    else:
        exchange = value_number
    return int(exchange)

df.total_money_raised = df.total_money_raised.apply(moneyRaise)

In [16]:
df_tech_raise = df[(df.total_money_raised > 1000000) & (df["Tech/Other"] == "Tech")]
df_tech_raise
gdf = gpd.GeoDataFrame(df_tech_raise, geometry=gpd.points_from_xy(df_tech_raise.longitude, df_tech_raise.latitude))
gdf.crs = {'init' :'epsg:4326'}
gdf.reset_index(drop=True, inplace=True)
gdf.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,Tech/Other,geometry
0,52cdef7c4bab8bd675297d91,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,...,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,Tech,POINT (-118.39306 34.09037)
1,52cdef7c4bab8bd675297d96,Gizmoz,gizmoz,http://www.crunchbase.com/company/gizmoz,http://gizmoz.com,,,None,web,NaN,...,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472,Tech,POINT (-122.16947 37.48413)
2,52cdef7c4bab8bd675297d95,StumbleUpon,stumbleupon,http://www.crunchbase.com/company/stumbleupon,http://www.stumbleupon.com,http://www.stumbleupon.com/blog,,stumbleupon,web,NaN,...,,,,San Francisco,CA,USA,37.775196,-122.419204,Tech,POINT (-122.41920 37.77520)
3,52cdef7c4bab8bd675297d99,Lala,lala,http://www.crunchbase.com/company/lala,http://www.lala.com,http://www.lala.com/#blog,http://www.lala.com/#blog,None,games_video,NaN,...,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369,Tech,POINT (-122.15437 37.45115)
4,52cdef7c4bab8bd675297d9c,MeetMoi,meetmoi,http://www.crunchbase.com/company/meetmoi,http://www.meetmoi.com,,,meetmoi,social,15.0,...,,,,New York City,NY,USA,40.757929,-73.985506,Tech,POINT (-73.98551 40.75793)


In [17]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [18]:
print(airports.crs)
print(gdf.crs)

NameError: name 'airports' is not defined

In [32]:
airports = gpd.read_file("./Input/ne_10m_airports/ne_10m_airports.shp")

In [ ]:
distances = gdf.geometry.distance(airports.geometry[1]).min()
distances
print(distances)

In [ ]:
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p.x,p.y))

v = {'lat': gdf.geometry[0].x, 'lon': gdf.geometry[0].x}

distance_airport = airports.geometry.apply(lambda p: distance(v['lat'], v['lon'], p.x, p.y))
distance_airport.min()

In [ ]:
airport.geometry[]

In [ ]:
mile_buffer = gdf.geometry.buffer(15*5280)

# m = folium.Map(location=[39.9526,-75.1652], zoom_start=11)
HeatMap(data=gdf[['latitude', 'longitude']], radius=15).add_to(m)
    
# Plot each polygon on the map
# GeoJson(mile_buffer.to_crs(epsg=4326)).add_to(m)

# Show the map
embed_map(m, 'm1.html')

In [33]:
coordenada1 = gdf.geometry[0]
coordenada2 = airports.geometry[0]

print(coordenada1)
print(coordenada2)

POINT (-118.393064 34.090368)
POINT (75.95707224036518 30.85035985617018)


In [36]:
from numpy import cos, sin, arcsin, sqrt
from math import radians
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c
    return km

haversine(coordenada1.x, coordenada1.y, coordenada2.x, coordenada2.y)

12630.925968205976

In [50]:
a = np.array(gdf.geometry)
distances = []
min_distances = []

for f in gdf.geometry:
    for e in airports.geometry:
        distances.append(haversine(f.x, f.y, e.x, e.y))
    min_distances.append(min(distances))
    distances = []

In [47]:
len(min_distances)

2063

In [52]:
haversine(coordenada1.x, coordenada1.y, coordenada2.x, coordenada2.y)

417.85776794113264

In [55]:
gdf["closest_airport_km"] = min_distances

In [58]:
gdf[gdf["closest_airport_km"] < 15]

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,zip_code,city,state_code,country_code,latitude,longitude,Tech/Other,geometry,closest_airport,closest_airport_km
4,52cdef7c4bab8bd675297d9c,MeetMoi,meetmoi,http://www.crunchbase.com/company/meetmoi,http://www.meetmoi.com,,,meetmoi,social,15.0,...,,New York City,NY,USA,40.757929,-73.985506,Tech,POINT (-73.98551 40.75793),9.731279,9.731279
5,52cdef7c4bab8bd675297d9d,Joost,joost,http://www.crunchbase.com/company/joost,http://joost.com,http://blog.joost.com,http://blog.joost.com/atom.xml,None,games_video,0.0,...,10011-6903,New York,NY,USA,40.746497,-74.009447,Tech,POINT (-74.00945 40.74650),11.982996,11.982996
7,52cdef7c4bab8bd675297d8e,Facebook,facebook,http://www.crunchbase.com/company/facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299.0,...,,Dublin,None,IRL,53.344104,-6.267494,Tech,POINT (-6.26749 53.34410),9.344689,9.344689
8,52cdef7c4bab8bd675297d8e,Facebook,facebook,http://www.crunchbase.com/company/facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299.0,...,10017,New York,NY,USA,40.755716,-73.979247,Tech,POINT (-73.97925 40.75572),9.267190,9.267190
10,52cdef7c4bab8bd675297da1,Plaxo,plaxo,http://www.crunchbase.com/company/plaxo,http://www.plaxo.com,http://blog.plaxo.com/,http://blog.plaxo.com/index.rdf,plaxo,web,50.0,...,94089,Sunnyvale,CA,USA,37.387845,-122.055197,Tech,POINT (-122.05520 37.38784),11.291626,11.291626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,52cdef7f4bab8bd67529c6b7,NanoInk,nanoink,http://www.crunchbase.com/company/nanoink,http://www.nanoink.net,,,NanoInk,biotech,NaN,...,60077,Skokie,IL,USA,42.041574,-87.749253,Tech,POINT (-87.74925 42.04157),14.761868,14.761868
2056,52cdef7f4bab8bd67529c6b4,OptiMedica,optimedica,http://www.crunchbase.com/company/optimedica,http://www.optimedica.com,,,optimedica,biotech,NaN,...,95054,Santa Clara,CA,USA,37.377798,-121.974454,Tech,POINT (-121.97445 37.37780),4.081995,4.081995
2057,52cdef7f4bab8bd67529c6b6,Tiny Speck,tiny-speck,http://www.crunchbase.com/company/tiny-speck,http://tinyspeck.com,,,slackhq,games_video,45.0,...,BC V6B 1H5,Vancouver,None,CAN,49.282455,-123.109217,Tech,POINT (-123.10922 49.28246),11.162973,11.162973
2059,52cdef7f4bab8bd67529c6e0,Celestial Semiconductor,celestial-semiconductor,http://www.crunchbase.com/company/celestial-se...,http://www.celestialsemi.cn,,,,semiconductor,NaN,...,95134,San Jose,CA,USA,37.386417,-121.929170,Tech,POINT (-121.92917 37.38642),1.881088,1.881088


In [62]:
import matplotlib
plt.plot(min_distances)

AttributeError: module 'matplotlib' has no attribute 'plot'

In [63]:
airports.shape

(891, 36)